# Segmenting and Clustering Neighbourhoods in Toronto - Part 3

#### Importing libraries

In [1]:
import numpy as np 

import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from geopy.geocoders import Nominatim 

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans


import folium 

print('Libraries imported.')

Libraries imported.


#### Buliding the Toronto dataframe

In [2]:
# Importing the Neighbourhoods table and erasing the ones without Borough assigned
html = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(html)[0] 
df = df[df.Borough != 'Not assigned']

# Importing the coordinates table
csv = 'https://cocl.us/Geospatial_data'
coord = pd.read_csv(csv)

# Merging both tables
Toronto = pd.merge(df, coord, on='Postal Code')

Toronto.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


#### Map of all the Neighbourhoods

In [3]:

map_toronto = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

for lat,lng,borough,neighbourhood in zip(Toronto['Latitude'],Toronto['Longitude'],Toronto['Borough'],Toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)
map_toronto

#### Clustering the neighbourhoods with K-Means Clustering

In [4]:
k=5
toronto_clustering = Toronto.drop(['Postal Code','Borough','Neighbourhood'],1)
kmeans = KMeans(n_clusters = k,random_state=0).fit(toronto_clustering)
Toronto.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto

,Cluster Labels,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,4,M3A,North York,Parkwoods,43.753259,-79.329656
1,4,M4A,North York,Victoria Village,43.725882,-79.315572
2,2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,0,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,2,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,1,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,3,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,4,M3B,North York,Don Mills,43.745906,-79.352188
8,4,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


#### Map with the clusters

In [5]:

# create map
map_clusters = folium.Map(location=[43.651070,-79.347015],zoom_start=10)

# set color scheme for the clusters
x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, neighbourhood, cluster in zip(Toronto['Latitude'], Toronto['Longitude'], Toronto['Neighbourhood'], Toronto['Cluster Labels']):
    label = folium.Popup(' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters